In [ ]:
import numpy as np
def projection_Condat(y, a = 1):
    """
    Parameters:
        y (array): Input vector to project
        a (float): Radius of the L1 ball (default: 1)
    
    Returns:
        numpy.ndarray: Projected vector x
    """
    # Converting input to numpy array
    y = np.asarray(y, dtype=np.float64)
    N = len(y)

    # Initialize variables
    v = [y[0]]
    v_tilde = []
    rho = y[0] - a

    # Process all the elements except the first one
    for n in range(1, N):
        if y[n] > rho:
            # Update rho with the current element
            rho += (y[n] - rho) / (len(v) + 1)
            if rho > y[n] - a:
                v.append(y[n])
            else:
                v_tilde.extend(v)
                v = [y[n]]
                rho = y[n] - a

    # Reprocess elements in v_tilde
    if v_tilde:
        for el in v_tilde:
            if el > rho:
                v.append(el)
                rho += (el - rho) / len(v)

    # Refine the set v
    changed = True
    while changed:
        changed = False
        for el in v.copy():
            if el <= rho:
                v.remove(el)
                if len(v) > 0:
                    rho += (rho - el) / len(v)
                else:
                    rho = -np.inf
                changed = True
                break
    # Compute the projection
    tau = rho
    x = np.maximum(y - tau, 0)

    return x

def projected_gradient(
    objective_fun,
    gradient_fun,
    projection_operator,
    x0,
    hyperparams={"max_iterations": 20, "tolerance": 1e-6}
):
    """
    Inputs:
        - objective_fun (callable): Function f(x) to minimize (or maximize if sign=-1).
        - gradient_fun (callable): Gradient of f(x).
        - projection_operator (callable): Projection onto the feasible set (e.g., L∞-ball).
        - x0 (np.ndarray): Initial feasible point.
        - hyperparams (dict): Dictionary with hyperparameters:
            - "max_iterations" (int): Maximum number of iterations.
            - "tolerance" (float): Tolerance on gradient norm.
    Outputs:
        - x_t (np.ndarray): Final solution.
        - t (int): Number of iterations performed.
        - history (dict): Contains 'objective' and 'gradient_norm'.
    """
    # Defining the parameters
    x_t = x0.copy().astype(np.float32)
    max_iterations = hyperparams["max_iterations"]
    tolerance = hyperparams["tolerance"]
    stepsize_0 = 1.0
    alpha = 0.5
    beta = 0.5

    # History trackers
    history = {'objective': [], 'gradient_norm': []}

    # Starting the Projected Gradient iterations
    for t in range(1, max_iterations + 1):

        # Compute objective and gradient at current point
        f_x = objective_fun(x_t)
        grad_t = gradient_fun(x_t)
        grad_norm = np.linalg.norm(grad_t)

        # Store history
        history['objective'].append(f_x)
        history['gradient_norm'].append(grad_norm)

        # Backtracking line search
        stepsize = stepsize_0
        while True:
            # Take a gradient step and project it back to the feasible set
            x_candidate = x_t - stepsize * grad_t
            x_projected = projection_operator(x_candidate)

            # Compute the actual step
            d_t = x_projected - x_t

            # Check the objective at the projected point
            f_candidate = objective_fun(x_projected)

            # Compare the new objective to a linear approximation
            if f_candidate <= f_x + alpha * np.dot(grad_t, d_t):
                break
            
            # Reduce the step size if the condition is not satisfied
            stepsize *= beta

            # If stepsize becomes too small, we stop the line search
            if stepsize < 1e-10:
                print(f"Line search failed to find a step size at iteration {t}")
                break
        
        # Check stopping condition on the projected gradient step
        if np.linalg.norm(d_t) < tolerance:
            print(f"Converged: projected gradient step below tolerance at iteration {t}")
            break

        # Compute the next step
        x_t = x_projected

    return x_t, t, history